Conectando ao MySQL

In [4]:
import mysql.connector

cnx = mysql.connector.connect(host='localhost', user='gpasciano', password= 'Pandas@2024')

print(cnx)

In [6]:
cursor = cnx.cursor()

In [14]:
cursor.execute("SHOW DATABASES")

for db in cursor:
    print(db)

('biblioteca',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


Lendo o arquivo CSV

In [8]:
import pandas as pd

df = pd.read_csv("tabela_livros.csv", index_col=0)

df

,authors,published_date,description
title,,,
Fundamentals of Data Engineering,"Joe Reis, Matt Housley",2022-06-22,Data engineering has grown rapidly in the past...
Azure Data Engineering Cookbook,"Nagaraj Venkatesan, Ahmad Osama",2022-09-26,Nearly 80 recipes to help you collect and tran...
Data Engineering with dbt,Roberto Zagni,2023-06-30,Use easy-to-apply patterns in SQL and Python t...
Model and Data Engineering,"Ladjel Bellatreche, Óscar Pastor, Jesús M. Alm...",2016-09-06,This book constitutes the refereed proceedings...
Data Engineering Best Practices,"Richard J. Schiller, David Larochelle",2024-10-11,Explore modern data engineering techniques and...
Data Engineering and Management,"Rajkumar Kannan, Frederic Andres",2012-01-16,This book constitutes the thoroughly refereed ...
Data Engineering for AI/ML Pipelines,"Venkata Karthik Penikalapati, Mitesh Mangaonkar",2024-10-18,DESCRIPTION Data engineering is the art of bui...
Artificial Intelligence and Multimedia Data Engineering,"Suman Kumar Swarnkar, Sapna Singh Kshatri, Vir...",2023-12-15,This book explains different applications of s...
Model and Data Engineering,"Christian Attiogbé, Sadok Ben Yahia",2021-06-14,This book constitutes the refereed proceedings...


In [9]:
df.reset_index(inplace=True)

df

,title,authors,published_date,description
0,Fundamentals of Data Engineering,"Joe Reis, Matt Housley",2022-06-22,Data engineering has grown rapidly in the past...
1,Azure Data Engineering Cookbook,"Nagaraj Venkatesan, Ahmad Osama",2022-09-26,Nearly 80 recipes to help you collect and tran...
2,Data Engineering with dbt,Roberto Zagni,2023-06-30,Use easy-to-apply patterns in SQL and Python t...
3,Model and Data Engineering,"Ladjel Bellatreche, Óscar Pastor, Jesús M. Alm...",2016-09-06,This book constitutes the refereed proceedings...
4,Data Engineering Best Practices,"Richard J. Schiller, David Larochelle",2024-10-11,Explore modern data engineering techniques and...
5,Data Engineering and Management,"Rajkumar Kannan, Frederic Andres",2012-01-16,This book constitutes the thoroughly refereed ...
6,Data Engineering for AI/ML Pipelines,"Venkata Karthik Penikalapati, Mitesh Mangaonkar",2024-10-18,DESCRIPTION Data engineering is the art of bui...
7,Artificial Intelligence and Multimedia Data En...,"Suman Kumar Swarnkar, Sapna Singh Kshatri, Vir...",2023-12-15,This book explains different applications of s...
8,Model and Data Engineering,"Christian Attiogbé, Sadok Ben Yahia",2021-06-14,This book constitutes the refereed proceedings...
9,Data Engineering with Databricks Cookbook,Pulkit Chadha,2024-05-31,Work through 70 recipes for implementing relia...


In [10]:
df.columns

Index(['title', 'authors', 'published_date', 'description'], dtype='object')

In [12]:
cursor.execute("CREATE DATABASE biblioteca")

Criando a Tabela no MySQL

In [13]:
cursor.execute("""CREATE TABLE IF NOT EXISTS biblioteca.tabela_livros_dados(
               title VARCHAR(300),
               authors VARCHAR(300),
               published_date DATE,
               description TEXT
               )""")

Inserindo os dados na tabela

In [15]:
for index, row in df.iterrows():
    sql = """
    INSERT INTO biblioteca.tabela_livros_dados (title, authors, published_date, description)
    VALUES (%s, %s, %s, %s)
    """
    values = (row["title"], row["authors"], row["published_date"], row["description"])
    cursor.execute(sql, values)


cnx.commit()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


In [23]:
try:
    # Consumir resultados pendentes, se houver
    cursor.fetchall()
except mysql.connector.errors.InterfaceError:
    # Nenhum resultado para consumir, tudo certo
    pass

# Agora feche o cursor
cursor.close()


True

Confirmando se os dados estão na tabela

In [24]:
# Cria um novo cursor para consulta
cursor = cnx.cursor()
cursor.execute("SELECT * FROM biblioteca.tabela_livros_dados")

# Consome os resultados
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

# Fecha o cursor após o uso
cursor.close()


('Fundamentals of Data Engineering', 'Joe Reis, Matt Housley', datetime.date(2022, 6, 22), "Data engineering has grown rapidly in the past decade, leaving many software engineers, data scientists, and analysts looking for a comprehensive view of this practice. With this practical book, you'll learn how to plan and build systems to serve the needs of your organization and customers by evaluating the best technologies available through the framework of the data engineering lifecycle. Authors Joe Reis and Matt Housley walk you through the data engineering lifecycle and show you how to stitch together a variety of cloud technologies to serve the needs of downstream data consumers. You'll understand how to apply the concepts of data generation, ingestion, orchestration, transformation, storage, and governance that are critical in any data environment regardless of the underlying technology. This book will help you: Get a concise overview of the entire data engineering landscape Assess data 

True

In [25]:
from tabulate import tabulate

# Cria um novo cursor para consulta
cursor = cnx.cursor()
cursor.execute("SELECT * FROM biblioteca.tabela_livros_dados")

# Obtém os nomes das colunas
colunas = [desc[0] for desc in cursor.description]

# Consome os resultados
resultados = cursor.fetchall()

# Exibe os resultados em formato de tabela
print(tabulate(resultados, headers=colunas, tablefmt="psql"))

# Fecha o cursor após o uso
cursor.close()


+---------------------------------------------------------+-----------------------------------------------------------------------------------+------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

True